# Lab 5: Copias superficiales y profundas. Diccionarios. 

## 1. Copias superficiales y profundas
En los lab anteriores hemos visto cómo crear, acceder y modificar listas en memoria. Otra operación muy común a realizar es la de copia. Hasta ahora has visto como copiar tipos básicos así:

In [1]:
#Inicializo la variable valor_1 a 25
valor_1 = 25
#Ahora COPIO el valor de valor_1 a valor_2, pero como ya vimos en su día en realidad lo que ocurre es que la variable valor_2 pasa a apuntar en memoria a la misma zona que valor_1. Esto lo podemos comprobar mediante el uso de la función built-in id:
valor_2 = valor_1
print('valor_1 vale ' + str(valor_1) + ' y está guardado en ' + str(id(valor_1)))
print('valor_2 vale ' + str(valor_2) + ' y está guardado en ' + str(id(valor_2)))
#Si reasigno valor_2, ahora apuntará a una nueva zona de memoria:
valor_2 = 4
print('Ahora valor_2 vale ' + str(valor_2) + ' y está guardado en ' + str(id(valor_2)))

valor_1 vale 25 y está guardado en 4448305264
valor_2 vale 25 y está guardado en 4448305264
Ahora valor_2 vale 4 y está guardado en 4448304592


Vemos que lo que ocurre es que el valor no se copia realmente, sino que las variables apuntan a uno u otro sitio dependiendo de su contenido. Mientras que este comportamiento, específico de Python, es muy conveniente en multitud de ocasiones, puede conducir a problemas cuando lo que copiamos son elementos mutables compuestos por otros elementos (como por ejemplo las listas).

Resulta que Python sólo va a realizar copias "reales" cuando sea necesario, por ejemplo cuando el programador se lo indique explícitamente. Vamos a ver porqué. Este es el mismo ejemplo de copia usando listas:

In [2]:
colours1 = ["red", "blue"]
colours2 = colours1
print(colours1)
['red', 'blue']
print(colours2)
['red', 'blue']
print(id(colours1),id(colours2))

colours2 = ["rouge", "vert"]
print(colours1)
['red', 'blue']
print(colours2)

print(id(colours1),id(colours2))


['red', 'blue']
['red', 'blue']
4484231112 4484231112
['red', 'blue']
['rouge', 'vert']
4484231112 4484591048


![copia-1](img/deep_copy_detailed1.png)
En el ejemplo una lista cualquiera es asignada a colours_1. Esta lista es una lista que llamaremos "superficial" o "plana" ya que no contiene otras listas anidadas. En el siguiente paso se asigna colours1 a colours2. Mediante el uso de la función **id** vemos que ambas variables apuntan a la misma zona de memoria, o en otras palabras, comparten el objeto lista formado por "red" y "blue".

Si asignamos una nueva lista a colours2, los valores en colours1 permanecen inalterados, como era de esperar. Lo que ocurre es que una nueva posición de memoria pasa a ser ocupada por la lista formada por "rouge" y "vert", ya que hemos creado una lista completamente nueva, y la variable colours_2 ahora apunta a dicha posición. 

Ahora vamos a ver qué ocurre si cambiamos un sólo elemento de alguna de las listas:

In [3]:
colours1 = ["red", "blue"]
colours2 = colours1
print(id(colours1),id(colours2))
colours2[1] = "green"
print(id(colours1),id(colours2))
print(colours1)
print(colours2)

4484611016 4484611016
4484611016 4484611016
['red', 'green']
['red', 'green']


![copia-2](img/deep_copy_detailed2.png)
Ahora lo que pasa es que cuando se asigna un nuevo elemento a la posición 2 de colours2, ¡el valor en colours1 también cambia automáticamente!! ¿Sabes por qué?

Lo que pasa es que no se está asignando un valor nuevo a ninguna de las variables, sino que simplemente se modifican posiciones de memorias a las que (resulta que) apuntaban las variables colours1 y colours2.

### 1.1 Copiando con el operador slice
Es posible copiar elementos usando el ya conocido operador slice evitando algunos de los efectos indeseados que acabamos de ver:

In [4]:
list1 = ['a','b','c','d']
list2 = list1[:]
print(id(list1),id(list2))
list2[1] = 'x'
print(list2)
print(list1)

4485374408 4485375816
['a', 'x', 'c', 'd']
['a', 'b', 'c', 'd']


Pero si la lista inicial contiene sublistas, vamos a tener problemas, ya que el operador slice va a copiar la referencia a dichas sublistas:

In [ ]:
lst1 = ['a','b',['ab','ba']]
lst2 = lst1[:]
print(id(list1),id(list2))
print(id(list1[2]),id(list2[2]))

La siguiente imagen representa lo que ocurre en el anterior ejemplo. The following diagram depicts the data structure after the copying. Vemos que tanto lst1[2] como lst2[2] apuntan a la misma sublista formada por "ab" y "ba".
![copia-3](img/deep_copy_detailed3.png)

Ahora, si asignamos un nuevo valor al elemento 0 o 1 de una de las dos lista, no habrá ningún efecto no deseado:

In [5]:
lst1 = ['a','b',['ab','ba']]
lst2 = lst1[:]
lst2[0] = 'c'
print(lst1)
print(lst2)

['a', 'b', ['ab', 'ba']]
['c', 'b', ['ab', 'ba']]


No problem: 
![copia-4](img/deep_copy_detailed4.png)

Pero si ahora cambio un elemento de la sublista la liamos:

In [ ]:
lst2[2][1] = 'd'
print(lst1)
print(lst2)

A pesar de haber modificado la lista lst2, lst1 también recibe dichos cambios!!! 😱😱😱
![copia-5](img/deep_copy_detailed5.png)

### Copiando usando la función deepcopy
Una solución a este problema es usar la función deepcopy del module copy, que realiza una copia profunda (elementos anidados incluidos) de una lista:

In [6]:
from copy import deepcopy

lst1 = ['a','b',['ab','ba']]
lst2 = deepcopy(lst1)
print(lst1)
print(lst2)

print(id(lst1), id(lst2))
print(id(lst1[0]), id(lst2[0]))
print(id(lst2[2]), id(lst1[2]))



['a', 'b', ['ab', 'ba']]
['a', 'b', ['ab', 'ba']]
4485375048 4485374984
4450408016 4450408016
4485204296 4485374920


Al usar la función **id** vemos que la sublista ha sido copiada, porque id(lst2[2]) es diferente de id(lst1[2]). Es interesante sin embargo que las cadenas no se copian (La copia no sólo es **profunda** sino además **inteligente**), lst1[0] y lst2[0] por un lado y lst1[1] y lst2[1] por otro referencian la misma cadena respectivamente. 
Esta es la situación después de copiar la lista:
![copia-6](img/deep_copy_detailed7.png)

Si ahora modificamos la lista anidada:

In [7]:
lst2[2][1] = "d"
lst2[0] = "c"
print(lst1)
print(lst2)

['a', 'b', ['ab', 'ba']]
['c', 'b', ['ab', 'd']]


La situación pasa a ser la siguiente:
![copia-7](img/deep_copy_detailed8.png)

## 2. Diccionarios

Para finalizar con las abstracciones de datos en Python vamos a ver los **dictionarios**. 
Conceptualmente, un diccionario es un conjunto desordenado de elementos (a diferencia de las listas, que están ordenadas). Los elementos del diccionario son parejas **clave-valor** y se separan entre ellos mediante comas. Además, los elementos pertenecientes a un mismo diccionario se delimitan mediante el uso de llaves {}. Tanto la clave como el valor han de ser tipos válidos en Python (int, string, etc.), con la salvedad de que en el caso de las claves han de ser tipos **inmutables**.

**Ojo! Los diccionarios NO son secuencias y tiene su propio tipo! Por lo tanto no soportan las mismas operaciones**

### 2.1 Acceder y modificar valores
Los elementos en los diccionarios, a diferencia de las listas, son accedidos mediante su clave y no su posición (ya que no hay orden). Cada elemento especificado por una clave es único en el diccionario (Tan sólo hay un elemento asociado a cada clave). Vamos a verlo con unos ejemplos:

In [8]:
#Definiendo diccionarios (En este caso diccionarios vacíos. Podemos usar las llaves o la palabra reservada "dict")
dic_vacio = {} 
dic_vacio_2 = dict()
print(dic_vacio)
print(dic_vacio_2)

{}
{}


In [9]:
#Inicializando diccionarios no vacíos
cliente_1 = {"nombre" : "Maria Gonzalez", "direccion" : "Rios Rosas 52", "saldo" : 500}
cliente_2 = {"nombre" : "Pedro Lopez", "direccion" : "Portugalete 10", "saldo" : 200}
print(cliente_1)
print(cliente_2)

{'nombre': 'Maria Gonzalez', 'direccion': 'Rios Rosas 52', 'saldo': 500}
{'nombre': 'Pedro Lopez', 'direccion': 'Portugalete 10', 'saldo': 200}


In [10]:
#Accediendo a los valores de un diccionario (corchetes)
print(cliente_1['nombre'])
print(cliente_2['saldo'])

Maria Gonzalez
200


In [11]:
#Ya que no hay orden no puedo hacer cosas como:
cliente_1[1]

KeyError: 1

In [12]:
#También obtendremos error si intentamos acceder al valor de una clave inexistente:
cliente_1['edad']

KeyError: 'edad'

In [13]:
#Modificando y actualizando diccionarios previamente declarados
#Vamos a usar la notación de corchetes:
cliente_1['edad'] = 25
cliente_2['edad'] = 34
cliente_1['direccion'] = "Gran Vía 25"
print(cliente_1)
print(cliente_2)

{'nombre': 'Maria Gonzalez', 'direccion': 'Gran Vía 25', 'saldo': 500, 'edad': 25}
{'nombre': 'Pedro Lopez', 'direccion': 'Portugalete 10', 'saldo': 200, 'edad': 34}


### 2.2 Eliminar pares
Una operación común que haremos sobre los diccionarios es la de eliminar un par clave-valor. Para ello usaremos la palabra reservada **del** o las operación **pop()**, que ya vimos en las listas, aunque ahora se define de manera ligeramente diferente o **popitem()**.

In [ ]:
#Eliminamos la clave edad y su valor correspondiente
del cliente_1['edad']
print(cliente_1)

#### pop()
En el caso de **pop()** (recuerda lo visto en listas), queremos eliminar un elemento del diccionario pero salvándolo a una variable sobre la que podamos operar con él. Recordemos que en las listas la operación **pop()** extraía el último elemento de la lista en cuestión pero entonces ¿Tiene sentido usar pop en un diccionario, cuando sabemos que no tiene orden?

La respuesta es que sí, ya que en este caso **pop()** no realiza exactamente lo mismo. Para un diccionario D y una clave k, *D.pop(k)* va a eliminar la clave k devolviendo el valor correspondiente:

In [ ]:
v = cliente_2.pop('edad')
print(v)
print(cliente_2)

In [ ]:
#La clave tendrá que existir u obtendremos un error:
cliente_2.pop('edad')

#### popitem()
El caso de **popitem()** es ligeramente diferente: no acepta ningún parámetro y elimina un elemento devolviendo el par clave-valor en forma de **tupla**. Esto es muy conveniente cuando queremos procesar y descartar los elementos de un diccionario uno a uno:

In [14]:
#El orden en el que se devuelven los elementos es arbitrario. 
#estado, capital es una tupla (Una nueva manera de usar las tuplas es usar una variable para guardar cada elemento de las mismas)
capitales = {"Valladolid":"Castilla y Leon", "Sevilla":"Andalucía", "Barcelona": "Cataluña", "Santiago de Compostela":"Galicia", "Vitoria-Gasteiz":"País Vasco", "Zaragoza":"Aragón"}
estado, capital = capitales.popitem()
otra_tupla = capitales.popitem()
print(estado, capital)
print(otra_tupla[0], otra_tupla[1])

Zaragoza Aragón
Vitoria-Gasteiz País Vasco


### 2.3 Comprobar si existe una clave
Hemos visto que si intentamos acceder a una clave inexistente en el diccionario obtendremos un error. Sin embargo hay una manera de evitar esto, comprobando si un diccionario contiene cierta clave usando los operadores **in** y **not in** que vimos en otros labs.

In [ ]:
if "Valladolid" in capitales:
    print('La capital de ' + capitales["Valladolid"] + ' es Valladolid')
if "Toledo" not in capitales:
          print("No se de qué comunidad autónoma es capital Toledo 😔")

Otra manera es usar **get()**, que no devuelve un error si no existe la clave, sino que devuelve None (palabra y tipo reservados).

In [15]:
print(capitales.get('Valladolid'))
print(capitales.get('Toledo'))

Castilla y Leon
None


### 2.4 Copiando diccionarios
Los diccionarios se pueden copiar usando el método **copy()**.
¡Cuidado! Se realiza una copia superficial, por lo que las modificaciones a elementos aninados siguen afectando a las copias.

In [23]:
cursos = { "curso1":{"titulo":"Introducción a Python", 
                         "localidad":"Madrid", 
                         "instructor":"Dr. Acula"},
              "curso2":{"titulo":"Python Avanzado",
                         "localidad":"Salamanca",
                         "instructor":"Abel Benito"},
              "curso3":{"titulo":"Procesamiento de Textos",
                         "localidad":"Zamora",
                         "instructor":"Armando Guerra"}
              }

cursos_copia = cursos.copy()
cursos["curso2"]["titulo"] = "Javascript Avanzado"
print(cursos_copia)

##Sin embargo si añadimos un nuevo elemento, funciona como esperamos
cursos["curso2"] = {"titulo":"Seminario de Latex para Principiantes",
                         "localidad":"Salamanca",
                         "instructor":"Yola Prieto"}

print(cursos)
print(cursos_copia)


{'curso1': {'titulo': 'Introducción a Python', 'localidad': 'Madrid', 'instructor': 'Dr. Acula'}, 'curso2': {'titulo': 'Javascript Avanzado', 'localidad': 'Salamanca', 'instructor': 'Abel Benito'}, 'curso3': {'titulo': 'Procesamiento de Textos', 'localidad': 'Zamora', 'instructor': 'Armando Guerra'}}
{'curso1': {'titulo': 'Introducción a Python', 'localidad': 'Madrid', 'instructor': 'Dr. Acula'}, 'curso2': {'titulo': 'Seminario de Latex para Principiantes', 'localidad': 'Salamanca', 'instructor': 'Yola Prieto'}, 'curso3': {'titulo': 'Procesamiento de Textos', 'localidad': 'Zamora', 'instructor': 'Armando Guerra'}}
{'curso1': {'titulo': 'Introducción a Python', 'localidad': 'Madrid', 'instructor': 'Dr. Acula'}, 'curso2': {'titulo': 'Javascript Avanzado', 'localidad': 'Salamanca', 'instructor': 'Abel Benito'}, 'curso3': {'titulo': 'Procesamiento de Textos', 'localidad': 'Zamora', 'instructor': 'Armando Guerra'}}


### 2.5 Limpiando
Por último, el diccionario puede ser limpiado (no borrado!) con el método **clear()**

In [19]:
cursos.clear()
print(cursos)

{}


### Más operaciones: concatenar e iterar:
De manera análoga a como concatenábamos listas, podemos hacer lo mismo con diccionarios usando el método **update(dict)**. Este método une las claves y valores de un diccionario, sobreescribiendo los valores para claves repetidas.

In [24]:
cursos.update({"curso2": {"titulo" : "Seminario de Javascript para Principiantes",
                         "localidad":"Valladolid",
                         "instructor":"Dolores Fuertes"}})

print(cursos)

{'curso1': {'titulo': 'Introducción a Python', 'localidad': 'Madrid', 'instructor': 'Dr. Acula'}, 'curso2': {'titulo': 'Seminario de Javascript para Principiantes', 'localidad': 'Valladolid', 'instructor': 'Dolores Fuertes'}, 'curso3': {'titulo': 'Procesamiento de Textos', 'localidad': 'Zamora', 'instructor': 'Armando Guerra'}}


Para iterar utilizaremos el ya conocido operador for ... in y las funciones keys() y values(), que devuelven listas con las claves y valores del diccionario, respectivamente.

In [26]:
for curso in cursos:
    print(curso)

print('#########')

for clave in cursos.keys():
    print(clave)
    
print('#########')
    
for value in cursos.values():
    print(value)

print('#########')

#Más lento
for clave in cursos.keys():
    print(cursos[clave])

print('#########')
#Tuplas clave-valor
for clave, valor in cursos.items():
    print(clave)
    print(valor)


curso1
curso2
curso3
#########
curso1
curso2
curso3
#########
{'titulo': 'Introducción a Python', 'localidad': 'Madrid', 'instructor': 'Dr. Acula'}
{'titulo': 'Seminario de Javascript para Principiantes', 'localidad': 'Valladolid', 'instructor': 'Dolores Fuertes'}
{'titulo': 'Procesamiento de Textos', 'localidad': 'Zamora', 'instructor': 'Armando Guerra'}
#########
{'titulo': 'Introducción a Python', 'localidad': 'Madrid', 'instructor': 'Dr. Acula'}
{'titulo': 'Seminario de Javascript para Principiantes', 'localidad': 'Valladolid', 'instructor': 'Dolores Fuertes'}
{'titulo': 'Procesamiento de Textos', 'localidad': 'Zamora', 'instructor': 'Armando Guerra'}
#########
curso1
{'titulo': 'Introducción a Python', 'localidad': 'Madrid', 'instructor': 'Dr. Acula'}
curso2
{'titulo': 'Seminario de Javascript para Principiantes', 'localidad': 'Valladolid', 'instructor': 'Dolores Fuertes'}
curso3
{'titulo': 'Procesamiento de Textos', 'localidad': 'Zamora', 'instructor': 'Armando Guerra'}
